In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
resp = requests.get('https://www.weather.go.kr/weather/observation/currentweather.jsp?type=t99&mode=0&stn=0&reg=100&auto_man=a')

In [3]:
resp.text[:200]

'<!DOCTYPE html>\n\n<html lang="ko">\n\t<head>\n\t\t<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n\t\t<title>도시별 현재날씨 &gt; 지상관측자료 &gt; 관측자료 &gt; 날씨 &gt; 기상청 </title>\n\t\t<link rel="shortcut icon" href="'

In [4]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [13]:
# 제목 행의 각 요소 값을 읽어서 컬럼 리스트에 저장

# soup.select("table.table_develop3 #table_header2 th")
# soup.select("table.table_develop3 thead tr:last-child th")
ths = soup.select("table.table_develop3 thead tr:nth-child(2) th")

columns = ['지점']
for th in ths:
    # print( th.get_text() )
    columns.append(th.get_text().strip())
columns

['지점',
 '현재일기',
 '시정km',
 '운량1/10',
 '중하운량',
 '현재기온',
 '이슬점온도',
 '체감온도',
 '일강수mm',
 '적설cm',
 '습도%',
 '풍향',
 '풍속',
 '해면기압']

In [28]:
# 데이터 행의 각 요소 값을 읽어서 행 리스트에 저장

rows = [] # 행의 목록

# soup.select('table.table_develop3 tbody tr')[:3]
trs = soup.select('table.table_develop3 tbody tr')
for tr in trs: # 행 반복
    # print( len(tr.select('td')) )

    row = [] # 1행
    tds = tr.select('td')
    for td in tds:
        # print(td.get_text(), end=" ")
        row.append(td.get_text().strip())
    # print()
    rows.append(row)

# rows

In [29]:
import pandas as pd

In [35]:
current_weather = pd.DataFrame(data=rows, columns=columns)

In [40]:
current_weather.head()
current_weather.sort_values(by='체감온도', ascending=False).head(10)

,지점,현재일기,시정km,운량1/10,중하운량,현재기온,이슬점온도,체감온도,일강수mm,적설cm,습도%,풍향,풍속,해면기압
3,거제,,8.3,,,10.0,-1.1,8.2,,,46,서북서,,1011.2
49,양산시,,10.2,,,10.8,-2.9,8.0,,,38,서,,1009.3
77,창원,구름많음,20 이상,7,7,9.3,-3.9,7.4,,,39,서북서,,1009.6
59,울산,구름조금,20 이상,4,4,10.0,-4.7,7.3,,,35,서,,1007.6
39,서귀포,연무,3.2,0,0,9.4,0.7,7.3,,,55,서,,1015.3
35,북창원,,20 이상,,,9.8,-0.9,7.1,,,47,서북서,,1010.0
31,부산,맑음,20 이상,0,0,10.6,-3.1,6.8,,,38,남서,,1008.2
86,통영,,9.8,,,9.4,0.2,6.8,,,53,서,,1011.6
15,김해시,,8.7,,,10.0,-3.6,6.7,,,38,북서,,1009.8
88,포항,구름많음,20 이상,8,5,9.3,-2.6,6.4,,,43,서,,1008.5


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
resp = requests.get('http://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2020&x=17&y=0&obs=07')

In [3]:
resp.text[:200]

'<!DOCTYPE html>\n\n<html lang="ko">\n\t<head>\n\t\t<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n\t\t<title>요소별자료 &gt; 과거자료 &gt; 지상관측자료 &gt; 관측자료 &gt; 날씨 &gt; 기상청 </title>\n\t\t<link rel="shortcut icon"'

In [4]:
soup = BeautifulSoup(resp.text, 'lxml')

In [5]:
tbl = soup.select_one('table.table_develop') # 한 개의 객체만 존재하므로 단일 객체 반환 함수 사용

In [6]:
# 헤더 정보 가져와서 컬럼 목록 만들기
columns = []
ths = tbl.select('thead tr th')
for th in ths:
    columns.append(th.get_text().strip())

columns[0] = '일'
columns

['일',
 '1월',
 '2월',
 '3월',
 '4월',
 '5월',
 '6월',
 '7월',
 '8월',
 '9월',
 '10월',
 '11월',
 '12월']

In [7]:
# tbody 하위의 tr 정보를 읽어서 행 데이터를 리스트로 구성
import numpy as np

rows = []
trs = tbl.select('tbody tr')
for tr in trs:
    row = []
    tds = tr.select('td')
    for td in tds:
        v = td.get_text().strip()
        row.append( np.nan if len(v) == 0 else v )
    rows.append(row)

rows = rows[:-1] # 마지막 행인 평균값 행을 제거

In [9]:
# rows

In [12]:
seoul_temp_2020 = pd.DataFrame(data=rows, columns=columns, dtype='float') # dtype -> 자료형을 float으로 변경 (가능하다면)

In [13]:
seoul_temp_2020.info()
seoul_temp_2020.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일       31 non-null     object 
 1   1월      31 non-null     float64
 2   2월      29 non-null     float64
 3   3월      31 non-null     float64
 4   4월      30 non-null     float64
 5   5월      31 non-null     float64
 6   6월      30 non-null     float64
 7   7월      31 non-null     float64
 8   8월      31 non-null     float64
 9   9월      30 non-null     float64
 10  10월     31 non-null     float64
 11  11월     30 non-null     float64
 12  12월     31 non-null     float64
dtypes: float64(12), object(1)
memory usage: 3.3+ KB


,일,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
26,27일,5.9,6.1,11.9,12.9,17.4,24.1,24.3,27.1,20.4,14.0,4.1,5.8
27,28일,6.8,4.5,7.7,13.9,18.7,25.2,24.4,27.3,20.7,13.8,-0.1,6.7
28,29일,6.0,7.7,8.7,14.9,20.7,23.8,23.7,26.8,19.6,10.8,-0.6,0.1
29,30일,5.2,NaN,10.2,17.1,22.6,20.1,25.3,26.8,19.6,11.9,-0.2,-10.9
30,31일,3.9,NaN,11.9,NaN,21.2,NaN,26.2,26.4,NaN,13.6,NaN,-8.9


In [14]:
seoul_temp_2020.set_index(['일'], inplace=True) # '일' 컬럼을 행 index로 변경

In [15]:
seoul_temp_2020.head()

,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
일,,,,,,,,,,,,
1일,-2.2,2.6,5.8,11.3,20.2,19.7,21.1,25.3,26.6,18.8,13.7,1.1
2일,1.0,1.5,3.4,9.3,20.3,17.9,24.2,24.9,24.3,19.2,10.8,2.4
3일,-0.1,-0.4,4.8,10.0,21.8,22.0,25.5,25.0,22.7,19.9,5.1,0.7
4일,1.2,-1.6,2.2,8.9,20.2,21.9,24.2,27.0,23.3,16.6,4.9,-0.6
5일,1.3,-8.3,1.4,7.2,16.0,23.0,23.8,26.5,23.3,13.3,9.6,2.1


In [16]:
seoul_temp_2020 = seoul_temp_2020.T # 행과 열의 위치를 교체

In [17]:
seoul_temp_2020

일,1일,2일,3일,4일,5일,6일,7일,8일,9일,10일,...,22일,23일,24일,25일,26일,27일,28일,29일,30일,31일
1월,-2.2,1.0,-0.1,1.2,1.3,3.1,6.3,2.4,-1.2,-0.6,...,3.0,4.6,2.8,4.3,5.5,5.9,6.8,6.0,5.2,3.9
2월,2.6,1.5,-0.4,-1.6,-8.3,-6.4,-1.7,-0.1,-0.5,4.0,...,4.0,2.5,7.0,6.7,6.4,6.1,4.5,7.7,NaN,NaN
3월,5.8,3.4,4.8,2.2,1.4,3.7,7.0,9.3,11.1,6.4,...,11.0,10.6,11.4,12.5,14.8,11.9,7.7,8.7,10.2,11.9
4월,11.3,9.3,10.0,8.9,7.2,9.3,10.6,9.4,8.9,10.7,...,6.0,7.9,9.9,13.1,12.3,12.9,13.9,14.9,17.1,NaN
5월,20.2,20.3,21.8,20.2,16.0,19.4,20.7,19.9,14.2,14.4,...,18.5,20.1,17.1,16.6,16.2,17.4,18.7,20.7,22.6,21.2
6월,19.7,17.9,22.0,21.9,23.0,24.6,22.7,25.0,26.7,25.8,...,28.6,27.3,22.3,21.6,21.8,24.1,25.2,23.8,20.1,NaN
7월,21.1,24.2,25.5,24.2,23.8,25.6,24.1,25.5,27.5,24.5,...,23.7,21.1,22.5,24.3,25.4,24.3,24.4,23.7,25.3,26.2
8월,25.3,24.9,25.0,27.0,26.5,24.1,24.3,24.2,24.9,26.5,...,24.3,25.3,27.8,29.9,30.2,27.1,27.3,26.8,26.8,26.4
9월,26.6,24.3,22.7,23.3,23.3,21.7,19.2,23.4,21.1,22.8,...,20.2,19.2,20.6,20.1,19.3,20.4,20.7,19.6,19.6,NaN
10월,18.8,19.2,19.9,16.6,13.3,14.2,16.0,17.2,17.2,16.9,...,13.5,8.6,8.7,11.8,12.9,14.0,13.8,10.8,11.9,13.6


In [40]:
# 수집된 데이터를 년-월-일 / 지역 / 기온 형태로 변형

# print( seoul_temp_2020.index, seoul_temp_2020.columns )
rows = []
for rname in seoul_temp_2020.index:
    for cname in seoul_temp_2020.columns:
        if not np.isnan(seoul_temp_2020.loc[rname, cname]):
            # print("{0}-{1}-{2}, {3}".format(2020, rname[:-1].zfill(2), cname[:-1].zfill(2), seoul_temp_2020.loc[rname, cname]))
            date = "{0}-{1}-{2}".format(2020, rname[:-1].zfill(2), cname[:-1].zfill(2)) # "1".zfill(3) -> "001"
            temp = seoul_temp_2020.loc[rname, cname]
            rows.append([date, '서울', temp])

# print( rows[:10] )
seoul_temp = pd.DataFrame(rows, columns=['date', 'region', 'temperature'])
seoul_temp

,date,region,temperature
0,2020-01-01,서울,-2.2
1,2020-01-02,서울,1.0
2,2020-01-03,서울,-0.1
3,2020-01-04,서울,1.2
4,2020-01-05,서울,1.3
...,...,...,...
361,2020-12-27,서울,5.8
362,2020-12-28,서울,6.7
363,2020-12-29,서울,0.1
364,2020-12-30,서울,-10.9
